In [1]:
import numpy as np
import tensorflow 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
structures = np.loadtxt("structures_120k.txt")

In [3]:
structures.shape

(120000, 8)

In [4]:
structures = structures/200

In [5]:
structures[:5]

array([[0.935, 0.52 , 0.52 , 0.84 , 0.795, 0.86 , 0.575, 0.505],
       [0.995, 0.85 , 0.59 , 0.125, 0.365, 0.695, 0.99 , 0.21 ],
       [0.665, 0.205, 0.82 , 0.485, 0.44 , 0.315, 0.63 , 0.185],
       [0.515, 0.835, 0.6  , 0.725, 0.585, 0.795, 0.53 , 0.58 ],
       [0.255, 0.885, 0.565, 0.27 , 0.4  , 0.87 , 0.825, 0.725]])

In [6]:
spectra = np.loadtxt("spectra_120k.dat")

In [7]:
spectra = spectra.reshape(120000,61,4)

In [8]:
spectra.shape

(120000, 61, 4)

In [9]:
structures_train,structures_test,spec_train,spec_test = structures[:110000],structures[110000:],spectra[:110000],spectra[110000:]

In [10]:
spec_test.shape

(10000, 61, 4)

In [11]:
k = 500  #here k is no clusters
n_samples = 5000  #no. of data points to select

In [12]:
kmeans = KMeans(n_clusters=k)
kmeans.fit(structures_train)

KMeans(n_clusters=500)

In [13]:
labels = kmeans.labels_

In [14]:
samples_per_cluster = n_samples // k

In [15]:
sampled_structures = []
sampled_spectra = []

In [16]:
for cluster in range(k):
    # Get the indices of data points in this cluster
    cluster_indices = np.where(labels == cluster)[0]
    
    # Randomly sample 'samples_per_cluster' from this cluster (without fixed seed)
    sampled_indices = np.random.choice(cluster_indices, samples_per_cluster, replace=False)
    
    # Append the sampled data points and their corresponding output spectra
    sampled_structures.append(structures_train[sampled_indices])
    sampled_spectra.append(spec_train[sampled_indices])

In [17]:
sampled_structures = np.vstack(sampled_structures)
sampled_spectra = np.vstack(sampled_spectra)

In [19]:
sampled_spectra.shape

(5000, 61, 4)

In [21]:
model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Input(shape=(8,)),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),

    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(256, activation='elu'),

    tensorflow.keras.layers.Reshape((32, 8)),
    tensorflow.keras.layers.Conv1D(256, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Reshape((16, 8)),
    tensorflow.keras.layers.Conv1D(128, kernel_size=5, padding='same', activation='elu'),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(128, activation='elu'),

    tensorflow.keras.layers.Dense(128, activation='elu'),
    tensorflow.keras.layers.Dense(244, activation='sigmoid'),
    tensorflow.keras.layers.Reshape((61, 4)),
])

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 32, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 256)        │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_4 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 16, 8)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 16, 128)        │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       262,27

 Total params: 2,708,212 (10.33 MB)

 Trainable params: 2,708,212 (10.33 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.001),loss='mse')

In [25]:
history =  model.fit(sampled_structures,sampled_spectra,epochs=100,batch_size=256,validation_split=0.3)


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.1107 - val_loss: 0.0316
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0294 - val_loss: 0.0260
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0258 - val_loss: 0.0253
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0250 - val_loss: 0.0249
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0253 - val_loss: 0.0249
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0250 - val_loss: 0.0247
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0247 - val_loss: 0.0243
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0243 - val_loss: 0.0237
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0239 - val_loss: 0.0235
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0237 - val_loss: 0.0232
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0236 - val_loss: 0.0230
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step

In [26]:
predicted_spectra_train = model.predict(structures_train)


3438/3438 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step


In [28]:
mse = tensorflow.keras.losses.MeanSquaredError()
loss = mse(spec_train, predicted_spectra_train).numpy()

print(f"The loss for 110k data of test1 is  {loss}")

The loss for 110k data of test1 is  0.016826007515192032


In [29]:
predicted_spectra_test = model.predict(structures_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [30]:
mne = tensorflow.keras.losses.MeanSquaredError()
losses = mse(spec_test,predicted_spectra_test).numpy()

print(f"The loss for 10k data of test1 is  {losses}")

The loss for 10k data of test1 is  0.01693662442266941
